# Structured Output

In [1]:
import os

import azure.identity
import openai
import rich
from dotenv import load_dotenv
from pydantic import BaseModel, Field

load_dotenv(override=True)
API_HOST = os.getenv("API_HOST", "github")

client = openai.OpenAI(
    base_url="https://models.github.ai/inference",
    api_key=os.environ["GITHUB_TOKEN"],
)

MODEL_NAME = os.getenv("GITHUB_MODEL", "openai/gpt-4o")

In [2]:
# define the schema for the structured output
class CalendarEvent(BaseModel):
    name: str
    date: str = Field(..., description="A date in the format YYYY-MM-DD") #it is recommended to add description for better parsing
    participants: list[str]


completion = client.beta.chat.completions.parse(
    model=MODEL_NAME,
    messages=[
        {
            "role": "system",
            "content": "Extract the event information. If no year is specified, assume the current year (2025).",
        },
        {"role": "user", "content": "Alice and Bob are going to a science fair on the 1st of april."},
    ],
    response_format=CalendarEvent,
)

In [4]:
message = completion.choices[0].message
# check if it's able to adhere to the schema
if message.refusal:
    rich.print(message.refusal)
else:
    event = message.parsed
    rich.print(event)

CalendarEvent(name='Science Fair', date='2025-04-01', participants=['Alice', 'Bob'])

## With Enum

In [5]:
from enum import Enum

In [6]:
class DayOfWeek(str, Enum):
    SUNDAY = "Sunday"
    MONDAY = "Monday"
    TUESDAY = "Tuesday"
    WEDNESDAY = "Wednesday"
    THURSDAY = "Thursday"
    FRIDAY = "Friday"
    SATURDAY = "Saturday"


class CalendarEvent(BaseModel):
    name: str
    date: DayOfWeek
    participants: list[str]

In [7]:
completion = client.beta.chat.completions.parse(
    model=MODEL_NAME,
    messages=[
        {"role": "system", "content": "Extract the event information."},
        {"role": "user", "content": "Alice and Bob are going to a science fair on Friday."},
    ],
    response_format=CalendarEvent,
)


message = completion.choices[0].message
if message.refusal:
    rich.print(message.refusal)
else:
    event = message.parsed
    rich.print(event)
    print(event.date.value)  # Access the enum value

CalendarEvent(name='Science Fair', date=<DayOfWeek.FRIDAY: 'Friday'>, participants=['Alice', 'Bob'])

Friday


## Nested data models

In [8]:
class Participant(BaseModel):
    name: str
    job_title: str


class CalendarEvent(BaseModel):
    name: str
    date: str
    participants: list[Participant]

In [9]:
completion = client.beta.chat.completions.parse(
    model=MODEL_NAME,
    messages=[
        {"role": "system", "content": "Extract the event information."},
        {"role": "user", "content": "Alice the carpenter and Bob the plumber are going to a science fair on Friday."},
    ],
    response_format=CalendarEvent,
)


message = completion.choices[0].message
if message.refusal:
    rich.print(message.refusal)
else:
    event = message.parsed
    rich.print(event)

CalendarEvent(
    name='Science Fair',
    date='Friday',
    participants=[Participant(name='Alice', job_title='Carpenter'), Participant(name='Bob', job_title='Plumber')]
)

## Function Calling

In [10]:
class GetDeliveryDate(BaseModel):
    order_id: str

In [11]:
response = client.chat.completions.create(
    model=MODEL_NAME,
    messages=[
        {"role": "system", "content": "You're a customer support bot. Use the tools to assist the user."},
        {"role": "user", "content": "Hi, can you tell me the delivery date for my order #12345?"},
    ],
    tools=[openai.pydantic_function_tool(GetDeliveryDate)],
)

rich.print(response.choices[0].message.tool_calls[0].function)

Function(arguments='{"order_id":"12345"}', name='GetDeliveryDate')